In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LayerNormalization
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import KFold
from sklearn import preprocessing, metrics

In [2]:
slug = pd.read_csv("220306_slugs_dataset.csv")

In [3]:
slug.head()

,loadCase,slugDen,slugLen,pockDen,pockLen,slugVel,basePer,baseAmp,basePha,simTime,out_oopDisp_mm_n2,out_supReac_kN_n3,out_supReac_kN_n4,out_verDisp_mm_n5,out_oopDisp_mm_n5,out_endBend_kNm_n6
0,00001_sd786_sl6_pd234_pl10_sv6_bp13_ba850_bp74,785.67,6.47,234.16,10.06,5.77,12.55,0.85,74.10,45.8,197.45,16.22,56.42,248.24,180.47,198.49
1,00002_sd705_sl10_pd190_pl11_sv7_bp12_ba800_bp59,705.03,10.36,190.44,11.25,7.42,11.63,0.80,59.11,44.0,186.85,15.97,58.86,262.33,178.72,238.07
2,00003_sd742_sl12_pd159_pl13_sv7_bp12_ba650_bp65,741.98,11.64,159.09,12.52,7.42,11.54,0.65,65.23,48.2,155.47,14.29,48.63,220.89,163.53,180.64
3,00004_sd732_sl9_pd168_pl10_sv10_bp11_ba670_bp68,731.88,8.82,167.88,9.84,9.52,11.48,0.67,67.85,30.8,169.47,15.18,48.72,188.47,163.76,233.31
4,00005_sd669_sl11_pd214_pl9_sv8_bp13_ba810_bp57,669.21,11.46,213.87,8.56,8.18,12.86,0.81,56.68,37.7,223.64,16.90,58.51,271.74,242.37,219.33


In [4]:
slug = slug.dropna()

In [5]:
#Shuffle the dataset
slug = slug.sample(frac=1).reset_index(drop=True)

In [6]:
#Select columns
slug_df = slug[["slugDen","slugLen","pockDen","pockLen","slugVel","basePer", "baseAmp", "basePha"]]

#Create new features
slug_df['densDiff'] = slug_df['slugDen'] - slug_df['pockDen']
slug_df['centrFor'] = slug_df['slugVel']**2 * slug_df['densDiff']

C:\Users\mania\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\mania\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
scaler = preprocessing.StandardScaler()
scaler = scaler.fit(slug_df)

In [8]:
#Create Targets
slug_tar = slug[["out_oopDisp_mm_n2", "out_oopDisp_mm_n5", "out_endBend_kNm_n6"]]

In [9]:
slug_tar.head()

,out_oopDisp_mm_n2,out_oopDisp_mm_n5,out_endBend_kNm_n6
0,191.21,191.13,192.14
1,152.15,144.15,135.66
2,175.46,166.23,165.32
3,137.66,133.85,165.28
4,174.36,166.37,228.65


In [10]:
#Scale independent variables
slug_train_df = np.array(scaler.transform(slug_df))

In [11]:
#First 40,000 used to train model, test on remainder
train_df = np.array(scaler.transform(slug_df[:40000]))
train_tar = np.array(slug_tar[:40000])
test_df = np.array(scaler.transform(slug_df[40000:]))
test_tar = np.array(slug_tar[40000:])

In [169]:
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(256, input_dim=n_inputs, activation='relu'))
    
    model.add(Dense(512, activation='relu'))
        
    model.add(Dense(256, activation='relu'))
    
    model.add(Dense(128, activation='relu'))
    
    model.add(Dense(64, activation='relu'))
    
    model.add(Dense(n_outputs, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
    return model

In [170]:
n_inputs, n_outputs = train_df.shape[1], train_tar.shape[1]

In [171]:
model = get_model(n_inputs, n_outputs)

In [172]:
model.fit(slug_train_df, slug_tar.values, epochs=100, batch_size=100)

Train on 49975 samples
Epoch 1/100
49975/49975 [==============================] - 2s 34us/sample - loss: 2916.3169 - mean_squared_error: 2916.3171
Epoch 2/100
49975/49975 [==============================] - 1s 26us/sample - loss: 606.1554 - mean_squared_error: 606.1551
Epoch 3/100
49975/49975 [==============================] - 1s 28us/sample - loss: 515.4928 - mean_squared_error: 515.4926
Epoch 4/100
49975/49975 [==============================] - 1s 29us/sample - loss: 427.9347 - mean_squared_error: 427.9345
Epoch 5/100
49975/49975 [==============================] - 1s 28us/sample - loss: 350.1869 - mean_squared_error: 350.1869
Epoch 6/100
49975/49975 [==============================] - 1s 28us/sample - loss: 311.1107 - mean_squared_error: 311.1106
Epoch 7/100
49975/49975 [==============================] - 1s 28us/sample - loss: 285.1039 - mean_squared_error: 285.1039
Epoch 8/100
49975/49975 [==============================] - 1s 28us/sample - loss: 266.1507 - mean_squared_error: 266.1506

Epoch 68/100
49975/49975 [==============================] - 1s 27us/sample - loss: 66.2501 - mean_squared_error: 66.2502
Epoch 69/100
49975/49975 [==============================] - 1s 27us/sample - loss: 60.9512 - mean_squared_error: 60.9513
Epoch 70/100
49975/49975 [==============================] - 1s 27us/sample - loss: 60.2436 - mean_squared_error: 60.2435
Epoch 71/100
49975/49975 [==============================] - 1s 27us/sample - loss: 57.5939 - mean_squared_error: 57.5939
Epoch 72/100
49975/49975 [==============================] - 1s 28us/sample - loss: 60.9207 - mean_squared_error: 60.9207
Epoch 73/100
49975/49975 [==============================] - 1s 27us/sample - loss: 59.6617 - mean_squared_error: 59.6617
Epoch 74/100
49975/49975 [==============================] - 1s 27us/sample - loss: 58.0585 - mean_squared_error: 58.0585
Epoch 75/100
49975/49975 [==============================] - 1s 27us/sample - loss: 59.1702 - mean_squared_error: 59.1702
Epoch 76/100
49975/49975 [======

In [173]:
#Use model to make predictions for test set
preds = model.predict(test_df)

In [174]:
preds

array([[186.71364, 199.40068, 288.21658],
       [201.21779, 187.22932, 214.32243],
       [181.9077 , 198.81194, 179.8568 ],
       ...,
       [187.39027, 209.10724, 204.3669 ],
       [207.43307, 235.24199, 291.07812],
       [229.57819, 231.11835, 258.1093 ]], dtype=float32)

In [175]:
test_tar

array([[184.81, 203.26, 276.91],
       [200.75, 187.45, 211.6 ],
       [180.7 , 202.46, 171.58],
       ...,
       [192.76, 203.26, 202.19],
       [211.73, 242.58, 285.11],
       [222.34, 234.51, 269.97]])

In [176]:
mae = metrics.mean_absolute_error(test_tar, preds)
mse = metrics.mean_squared_error(test_tar, preds)
mape = metrics.mean_absolute_percentage_error(test_tar, preds)
rmse = np.sqrt(mse) # or mse**(0.5)  
r2 = metrics.r2_score(test_tar, preds)

In [177]:
print("MAE:", round(mae, 2))
print("MSE:", round(mse, 2))
print("MAPE:", round(mape, 2))
print("RMSE:", round(rmse, 2))
print("R-Squared:", round(r2, 2))

MAE: 4.7
MSE: 39.97
MAPE: 0.02
RMSE: 6.32
R-Squared: 0.98


In [178]:
mae = metrics.mean_absolute_error(test_tar[:,0], preds[:,0])
mse = metrics.mean_squared_error(test_tar[:,0], preds[:,0])
mape = metrics.mean_absolute_percentage_error(test_tar[:,0], preds[:,0])
rmse = np.sqrt(mse) # or mse**(0.5)  
r2 = metrics.r2_score(test_tar[:,0], preds[:,0])

In [179]:
print("Target 1")
print("MAE:", round(mae, 2))
print("MSE:", round(mse, 2))
print("MAPE:", round(mape, 2))
print("RMSE:", round(rmse, 2))
print("R-Squared:", round(r2, 2))

Target 1
MAE: 3.09
MSE: 16.18
MAPE: 0.02
RMSE: 4.02
R-Squared: 0.98


In [180]:
mae = metrics.mean_absolute_error(test_tar[:,1], preds[:,1])
mse = metrics.mean_squared_error(test_tar[:,1], preds[:,1])
mape = metrics.mean_absolute_percentage_error(test_tar[:,1], preds[:,1])
rmse = np.sqrt(mse) # or mse**(0.5)  
r2 = metrics.r2_score(test_tar[:,1], preds[:,1])

In [181]:
print("Target 2")
print("MAE:", round(mae, 2))
print("MSE:", round(mse, 2))
print("MAPE:", round(mape, 2))
print("RMSE:", round(rmse, 2))
print("R-Squared:", round(r2, 2))

Target 2
MAE: 4.51
MSE: 33.68
MAPE: 0.02
RMSE: 5.8
R-Squared: 0.98


In [182]:
mae = metrics.mean_absolute_error(test_tar[:,2], preds[:,2])
mse = metrics.mean_squared_error(test_tar[:,2], preds[:,2])
mape = metrics.mean_absolute_percentage_error(test_tar[:,2], preds[:,2])
rmse = np.sqrt(mse) # or mse**(0.5)  
r2 = metrics.r2_score(test_tar[:,2], preds[:,2])

In [183]:
print("Target 3")
print("MAE:", round(mae, 2))
print("MSE:", round(mse, 2))
print("MAPE:", round(mape, 2))
print("RMSE:", round(rmse, 2))
print("R-Squared:", round(r2, 2))

Target 3
MAE: 6.49
MSE: 70.04
MAPE: 0.03
RMSE: 8.37
R-Squared: 0.98
